In [19]:
from bs4 import BeautifulSoup
import pandas
import urllib, requests
from requests.auth import HTTPBasicAuth, HTTPDigestAuth
import re
# import nbimporter #need to pip install this
# import EncapsulatedSchoolObject as school

## Keyword lists

In [33]:
#mission = [' vision ', 'Vision ', 'our purpose', ' cause ', ' objectives ', ' mission', ' vision: ', ' mission: ', 'Mission']
#curriculum = [ 'program', 'methods', 'pedagogy', 'approach', 'model', 'curriculum', 'academics', 'degree']
#philosophy = ['beliefs', 'principles', 'creed', 'Values', 'philosophy', 'moral']
#history = ['story', 'background', 'founded', 'established']
#target = ['gifted', 'at-risk', 'ethnic background', 'target population', 'target audience']
#resources = ['ESL', 'tutoring', 'after-school programs', 'available resources', 'services', 'opportunities', 'opportunity']
#orgfactor = ['statistics', 'API', 'environment', 'ratio', 'average', 'female', 'fund', 'community']

mission = ['mission',' vision ', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective']
curriculum = ['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure']
philosophy = ['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value',  'moral', 'Values']
history = ['history', 'our story', 'the story', 'school story', 'background', 'founding', 'founded', 'established', 'establishment', 'our school began', 'we began', 'doors opened', 'school opened']
general =  ['about us', 'our school', 'who we are', 'overview', 'general information', 'our identity', 'profile', 'highlights']

keyValues = [mission, curriculum, philosophy, history, general]
nodeParams = ['mission', 'curriculum', 'philosophy', 'history', 'general']

In [24]:
# read in data
df = pandas.read_excel('micro-sample_MANUAL_Apr17_rev3.xlsx')

In [25]:
#get the values from the URL column (stored as list)
links = df['URL'].values

In [13]:
# Input test URL
url = links[0]
url

'https://www.richland2.org/charterhigh/'

In [29]:
# Create HTML soup
html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "lxml")

## BETA/ FOR REFERENCE (mostly from this notebook)

In [22]:
# AKCAN'S VERSION:
html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "lxml")

# THAO'S VERSION:
html_page = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
soup = BeautifulSoup(html_page.text, "lxml")

In [9]:
#(self, url, nodeParams, keyValues)


#creating a list of all the keyword lists
#keyValues = [mission, curriculum, philosophy, history, target, resources, orgfactor]
keyValues = [mission, curriculum, philosophy, history, general]

#creating a list of key names for the JSON node that correspond to the keyValues list
#nodeParams = ['mission', 'curriculum', 'philosophy', 'history', 'target', 'resources' , 'organizational_factors']
nodeParams = ['mission', 'curriculum', 'philosophy', 'history', 'general']
testedLinks = {}

#print(links[0])



#constructor = school.WebsiteData('https://www.richland2.org/charterhigh/', nodeParams, keyValues)
#constructor.createJsonNode()
#print(constructor.URL)

def tester(links, keyValues, nodeParams):
    testedLinks[links[5]] = school.WebsiteData(links[5], nodeParams, keyValues).createJsonNode()
    #for i in range(5, 6):
        #testedLinks[links[i]] = school.WebsiteData(links[i], nodeParams, keyValues).createJsonNode()
tester(links, keyValues, nodeParams)

NameError: name 'school' is not defined

## TESTING (from EncapsulatedSchoolObject.ipynb)

In [34]:
#Finds all tags within a BeautifulSoup object that's text contains a certain keyword and returns a list of these tags.
def findTags(soup, keyWord):
    tagList = []
        
    for elem in soup(text=re.compile(keyWord)):
        tagList.append(elem.parent)
            
    return tagList

In [59]:
#Utilizes a BeautifulSoup object (soup) to scrape all text that contains the given keyword. Filters for certain tags and does
#not take text from solely header (h) tags, instead takes text from next filtered tag after the header.  
def keyWordScraper(soup, keyWord):
    searchTxt = ''
    correctName = ['p', 'li', 'table', 'ul', 'ol']
    tagList = findTags(soup, keyWord)
    #print(tagList)
    
    #This loop goes through each tag and rips the text from that tag. If that tag is a header, then the text from the next 
    #useful tag is ripped instead. All text is saved in searchTxt
    for tag in tagList:
        #print(tag, "\n")
        if 'h' in tag.name:
            typeTag = type(soup.find('li'))
            current = tag.next_sibling
            while current != None and  not isinstance(current, typeTag) and current.name not in correctName:
                current = current.next_sibling
            if current != None:
                if 'ul' in current.name or 'ol' in current.name:
                    for li in current.findAll('li'):
                        searchTxt = searchTxt + li.text
                else:
                    searchTxt = searchTxt + current.text
        #elif url extension includes the keyWord, grab all the text?
        else:
            if(tag.name in correctName):
                    searchTxt = searchTxt + tag.text
                
        return searchTxt

In [63]:
#Applies the keyWordScraper function to a BeautifulSoup object(soup) for a list of keywords. Returns the text found.
def mapKeywordScraper(soup, keyWords):
    foundTxt = ''
    
    for keyWord in keyWords:
        foundTxt = foundTxt + str(keyWordScraper(soup, keyWord))
    #print(foundTxt)
    return foundTxt

In [61]:
def createJsonNode():
    JSONNode = {}
    
    for x in range(0, len(keyValues)):
        print(keyValues[x], "\n")
        JSONNode[nodeParams[x]] = [mapKeywordScraper(soup, keyValues[x])]
        print(JSONNode)
    return JSONNode

In [62]:
createJsonNode()

['mission', ' vision ', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective'] 

NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone
{'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone']}
['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure'] 

NoneNoneNoneNoneNoneNoneNoneNoneNoneNone
{'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone'], 'curriculum': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone']}
['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value', 'moral', 'Values'] 

NoneNoneNoneNoneNoneNoneNoneNoneNone
{'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone'], 'curriculum': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone'], 'philosophy': ['NoneNoneNoneNoneNoneNoneNoneNoneNone']}
['history', 'our story', 'the story', 'school story', 'background', 'founding', 'founded', 'established', 'establishment', 'our school began',

{'curriculum': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone'],
 'general': ['NoneNoneNoneNoneNoneNoneNoneNone'],
 'history': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone'],
 'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone'],
 'philosophy': ['NoneNoneNoneNoneNoneNoneNoneNoneNone']}

## CHUNKS OF CODE

In [13]:
# Input test URL here
url = links[0]
url

'https://www.richland2.org/charterhigh/'

In [29]:
html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "lxml")

In [30]:
def findTags(soup, keyWord):
    tagList = []
        
    for elem in soup(text=re.compile(keyWord)):
        tagList.append(elem.parent)
            
    return tagList

In [32]:
tagList = findTags(soup, keyWord)

TypeError: unhashable type: 'list'

In [ ]:
searchTxt = ''
correctName = ['p', 'li', 'table', 'ul', 'ol']
tagList = findTags(soup, keyWord)
    
#This loop goes through each tag and rips the text from that tag. If that tag is a header, then the text from the next 
 #useful tag is ripped instead. All text is saved in searchTxt
for tag in tagList:
    if 'h' in tag.name:
        typeTag = type(soup.find('li'))
        current = tag.next_sibling
        while current != None and  not isinstance(current, typeTag) and current.name not in correctName:
            current = current.next_sibling
        if current != None:
            if 'ul' in current.name or 'ol' in current.name:
                for li in current.findAll('li'):
                    searchTxt = searchTxt + li.text
            else:
                searchTxt = searchTxt + current.text
    #elif url extension includes the keyWord, grab all the text?
    else:
        if(tag.name in correctName):
            searchTxt = searchTxt + tag.text
                
return searchTxt